# PREPARE
You don't need mongodb to run this notebook. You can just use ConversaionBufferMemmory instead. This is for testing purpose only.

To be able to run the test in this notebook, you need some file in `CompassUTD\_secret\` folder. The folder should be in the same directory as this notebook. The files are:

`google_key.json`: - Google API key for Google Cloud Platform. You can get one by making a service account
`search_secrets.json`: - Google Custom Search API key and search engine ID. You can get one by making a custom search engine with these settings:
```
{
    "GOOGLE_SEARCH_API": "REPlACE_WITH_YOUR_KEY",
    "COURSE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "DEGREE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "RANDOM_SEARCH_ID": "REPlACE_WITH_YOUR_KEY"
  }
```



In [1]:
import os
from dotenv import load_dotenv

import random
import string

load_dotenv()

from CompassUTD import CompassInference
from langchain.memory import ReadOnlySharedMemory, MongoDBChatMessageHistory, ConversationBufferMemory,ConversationBufferWindowMemory


agent = CompassInference()




## MongoDB Test

In [2]:
# Use MongoDB
connection_string = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}"

random_letter_string = "".join([random.choice(string.ascii_letters) for _ in range(16)])

message_history = MongoDBChatMessageHistory(
        connection_string=connection_string, session_id= random_letter_string
    )

In [6]:

memmory_store = ConversationBufferWindowMemory(memory_key="chat_history",chat_memory=message_history, k = 4)

read_only_memory = ReadOnlySharedMemory(memory=memmory_store)


#If you want to make a new chat query with same memory, replace the user_message with next message and run this cell
user_message = "Which professor do research in AI"

bot_message = agent.run(user_message = user_message, 
                        read_only_memory= ReadOnlySharedMemory(memory=read_only_memory)
                        )

#Save chat history
message_history.add_user_message(user_message)
message_history.add_ai_message(bot_message)




> Entering new  chain...
Thought: I need to find professors that do research in AI
Action: general_search
Action Input: AI
Observation: {'data': " The Center for Applied AI and Machine Learning – The Center for Applied AI and Machine Learning at The University of Texas at Dallas Computer Science Department Home – main WHAT WE DOWHO WE ARE WHAT WE DO Applied AI And Machine Learning Bridge developments in AI and Machine Learning to real applications and services in collaboration with our industry partners. Technology Development And Deployment Develop new on-demand technologies for machine learning and large-scale data analysis. In-house Training Train a new generation of industrial scientists to address the skills shortage in AI and machine learning areas and increase competitiveness in the high-value information technology sectors, with the development of dedicated education programs. IntellectualProperty Develop IP with and for our partners on demand. Joint ResearchAnd Development J